In [1]:
# ===== Cell 1: 安裝與導入套件 =====
# 如果缺少套件，請先執行：
!pip install pandas numpy matplotlib seaborn plotly scipy scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
import glob
import warnings
from pathlib import Path
from datetime import datetime
from scipy import stats
import json

warnings.filterwarnings('ignore')

# 設定中文字體（Mac/Linux）
# plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'DejaVu Sans']
# Windows 用戶改用：
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
plt.rcParams['axes.unicode_minus'] = False

# 設定視覺化風格
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("✅ 套件載入完成")
print(f"📍 當前工作目錄: {os.getcwd()}")


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ 套件載入完成
📍 當前工作目錄: c:\Users\user\Desktop\step 1 +345 複製\workspace - others\orkspace


In [2]:
# ===== Cell 2: 建立專案資料夾結構 =====

def create_project_structure():
    """建立專案所需的資料夾結構"""
    folders = [
        'data/raw',
        'data/processed',
        'reports/figures/raw_timeseries',
        'reports/figures/delta_timeseries',
        'reports/figures/arduino_features',  # 👈 新增：Arduino 特徵視覺化
        'reports/figures/distributions',
        'reports/figures/stability',
        'reports/figures/correlation',
        'reports/figures/sensor_health',     # 👈 新增：感測器健康度
        'reports/summary'
    ]
    
    for folder in folders:
        Path(folder).mkdir(parents=True, exist_ok=True)
    
    print("✅ 資料夾結構建立完成：")
    for folder in folders:
        print(f"   📁 {folder}")

create_project_structure()

✅ 資料夾結構建立完成：
   📁 data/raw
   📁 data/processed
   📁 reports/figures/raw_timeseries
   📁 reports/figures/delta_timeseries
   📁 reports/figures/arduino_features
   📁 reports/figures/distributions
   📁 reports/figures/stability
   📁 reports/figures/correlation
   📁 reports/figures/sensor_health
   📁 reports/summary


In [3]:
!pip install openpyxl


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# 測試讀取一個檔案，看看真正的欄位名稱
import pandas as pd
import glob
import os

# 隨便讀一個檔案
test_file = glob.glob('data/raw/*.xlsx')[0]
df_test = pd.read_excel(test_file)

print("📋 實際的欄位名稱：")
print(df_test.columns.tolist())


📋 實際的欄位名稱：
['# ### MODE: PINEAPPLE_INTERNAL_ADC ###', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24']


In [5]:
#發現第一行是註解，所以要寫跳過

In [6]:
# ===== Cell 3: 資料載入器 (Excel 版本) - 自動偵測格式 =====

import pandas as pd
import glob
import os

class PineappleDataLoader:
    """批次載入與解析鳳梨感測器數據 (Excel 格式)"""
    
    def __init__(self, data_dir='data/raw'):
        self.data_dir = data_dir
        # air_data：key = "鳳梨編號_日期"（例如 01_20250205），value = DataFrame
        self.air_data = {}      
        # pineapple_data：{pineapple_id: {date: DataFrame}}
        self.pineapple_data = {}
        
    def load_all_data(self):
        """自動載入所有 Excel 檔案"""
        xlsx_files = glob.glob(os.path.join(self.data_dir, '*.xlsx'))
        
        if not xlsx_files:
            print(f"❌ 在 {self.data_dir} 找不到 Excel 檔案")
            return
        
        print(f"🔍 找到 {len(xlsx_files)} 個 Excel 檔案")
        
        for file_path in xlsx_files:
            filename = os.path.basename(file_path)
            self._parse_and_load(file_path, filename)
        
        print(f"\n✅ 載入完成:")
        print(f"   🌬️  Air baseline 檔案數: {len(self.air_data)}")
        print(f"   🍍 Pineapple: {len(self.pineapple_data)} 顆")
        
        return self.air_data, self.pineapple_data
    
    def _parse_and_load(self, file_path, filename):
        """解析單一檔案
        
        命名規則：
        - air: pineapple_01_20250205_air.xlsx
        - 鳳梨: pineapple_01_20250205.xlsx
        其中 01 為鳳梨編號，20250205 為日期
        """
        try:
            parts = filename.replace('.xlsx', '').split('_')
            
            if len(parts) < 3:
                print(f"⚠️  跳過不符合命名規則的檔案: {filename}")
                return
            
            pineapple_id = parts[1]   # '01'
            date = parts[2]           # '20250205'
            is_air = 'air' in filename
            
            # 讀取 Excel
            df = self._read_excel_with_validation(file_path, filename)
            
            if df is None:
                return
            
            if is_air:
                # 對應命名規則：不同日期的 air 會有不同 key
                air_key = f"{pineapple_id}_{date}"
                self.air_data[air_key] = df
                print(f"   ✓ Air: {filename} ({len(df)} rows) -> key={air_key}")
            else:
                if pineapple_id not in self.pineapple_data:
                    self.pineapple_data[pineapple_id] = {}
                self.pineapple_data[pineapple_id][date] = df
                print(f"   ✓ Pineapple: {filename} ({len(df)} rows)")
                
        except Exception as e:
            print(f"❌ 載入失敗 {filename}: {e}")
    
    def _read_excel_with_validation(self, file_path, filename):
        """
        讀取並驗證 Excel 格式
        自動偵測：
        1. 第 1 列就是欄位名稱（新格式）
        2. 第 1 列是註解，第 2 列才是欄位名稱（舊格式）
        """
        required_cols = [
            'timestamp_ms', 'MQ2_raw', 'MQ3_raw', 'MQ9_raw',
            'MQ135_raw', 'TGS2602_raw', 'Temp_C', 'Humidity_pct', 'Pressure_hPa'
        ]

        try:
            # ① 先試：第一列就是欄位名稱（新格式）
            df = pd.read_excel(file_path, engine='openpyxl')
            
            # 檢查第一列是不是註解
            first_col_name = str(df.columns[0])
            
            if first_col_name.startswith('#') or 'MODE' in first_col_name or 'Unnamed' in first_col_name:
                # 第一列是註解，跳過重讀
                df = pd.read_excel(file_path, engine='openpyxl', skiprows=1)
            
            # 驗證欄位
            missing_cols = [col for col in required_cols if col not in df.columns]
            
            if missing_cols:
                print(f"⚠️  {filename} 缺少欄位: {missing_cols}")
                print(f"      實際欄位前 5 個: {df.columns.tolist()[:5]}")
                return None
            
            # 強制轉換資料型別（避免字串問題）
            numeric_cols = ['timestamp_ms', 'MQ2_raw', 'MQ3_raw', 'MQ9_raw', 
                           'MQ135_raw', 'TGS2602_raw', 'Temp_C', 'Humidity_pct', 'Pressure_hPa']
            
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')  # 轉換成數字，無法轉的變 NaN
            
            return df

        except Exception as e:
            print(f"❌ 讀取錯誤 {filename}: {e}")
            return None

# 測試載入
loader = PineappleDataLoader('data/raw')
air_data, pineapple_data = loader.load_all_data()

# 顯示載入詳情
print("\n📊 載入詳情:")
print(f"Air baseline keys: {list(air_data.keys())}")
print(f"Pineapple IDs: {list(pineapple_data.keys())}")
for pid, dates in pineapple_data.items():
    print(f"  🍍 {pid}: {list(dates.keys())}")




🔍 找到 152 個 Excel 檔案
   ✓ Pineapple: pineapple_01_20250201.xlsx (912 rows)
   ✓ Air: pineapple_01_20250201_air.xlsx (912 rows) -> key=01_20250201
   ✓ Pineapple: pineapple_01_20250202.xlsx (912 rows)
   ✓ Air: pineapple_01_20250202_air.xlsx (912 rows) -> key=01_20250202
   ✓ Pineapple: pineapple_01_20250203.xlsx (912 rows)
   ✓ Air: pineapple_01_20250203_air.xlsx (912 rows) -> key=01_20250203
   ✓ Pineapple: pineapple_01_20250204.xlsx (912 rows)
   ✓ Air: pineapple_01_20250204_air.xlsx (912 rows) -> key=01_20250204
   ✓ Pineapple: pineapple_01_20250205.xlsx (912 rows)
   ✓ Air: pineapple_01_20250205_air.xlsx (912 rows) -> key=01_20250205
   ✓ Pineapple: pineapple_01_20250206.xlsx (912 rows)
   ✓ Air: pineapple_01_20250206_air.xlsx (912 rows) -> key=01_20250206
   ✓ Pineapple: pineapple_01_20250207.xlsx (912 rows)
   ✓ Air: pineapple_01_20250207_air.xlsx (912 rows) -> key=01_20250207
   ✓ Pineapple: pineapple_01_20250208.xlsx (912 rows)
   ✓ Air: pineapple_01_20250208_air.xlsx (912 rows)

In [7]:
# ===== Cell 4: 資料清洗器 + 感測器健康度檢查 =====

class DataCleaner:
    """資料清洗與驗證"""
    
    def __init__(self):
        # 定義異常值範圍（根據你的 Arduino code）
        self.validation_rules = {
            'MQ2_raw': (10, 1000),
            'MQ3_raw': (10, 1000),
            'MQ9_raw': (10, 1000),
            'MQ135_raw': (10, 1000),
            'TGS2602_raw': (10, 1000),
            'Temp_C': (0, 50),
            'Humidity_pct': (0, 100),
            'Pressure_hPa': (800, 1100)
        }
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
    
    def clean_dataset(self, df, dataset_name="Dataset"):
        """清洗單一數據集"""
        print(f"\n🧹 清洗 {dataset_name}...")
        
        original_rows = len(df)
        
        # 1. 移除完全重複的行
        df = df.drop_duplicates()
        
        # 2. 檢查並處理缺值
        missing_before = df.isnull().sum().sum()
        if missing_before > 0:
            print(f"   ⚠️  發現 {missing_before} 個缺值")
            df = df.dropna().reset_index(drop=True)  # 👈 關鍵：重置 index
        
        # 3. 異常值過濾
        mask = pd.Series([True] * len(df))
        outlier_counts = {}
        
        for col, (min_val, max_val) in self.validation_rules.items():
            if col in df.columns:
                col_mask = (df[col] >= min_val) & (df[col] <= max_val)
                outliers = (~col_mask).sum()
                if outliers > 0:
                    outlier_counts[col] = outliers
                mask &= col_mask
        
        df_clean = df[mask].copy()
        
        # 4. 時間對齊（重設 timestamp_s 從 0 開始）
        if 'timestamp_ms' in df_clean.columns:
            df_clean['timestamp_s'] = (df_clean['timestamp_ms'] - df_clean['timestamp_ms'].iloc[0]) / 1000.0
        
        # 5. 感測器健康度檢查
        health_report = self.check_sensor_health(df_clean, dataset_name)
        
        # 報告
        removed_rows = original_rows - len(df_clean)
        print(f"   ✓ 原始: {original_rows} rows")
        print(f"   ✓ 移除: {removed_rows} rows ({removed_rows/original_rows*100:.2f}%)")
        print(f"   ✓ 保留: {len(df_clean)} rows")
        
        if outlier_counts:
            print(f"   📊 異常值統計:")
            for col, count in outlier_counts.items():
                print(f"      - {col}: {count}")
        
        return df_clean, {
            'original_rows': original_rows,
            'removed_rows': removed_rows,
            'outlier_counts': outlier_counts,
            'final_rows': len(df_clean),
            'sensor_health': health_report
        }
    
    def check_sensor_health(self, df, dataset_name):
        """檢查感測器健康度"""
        print(f"   🔬 感測器健康度:")
        
        health_report = {}
        
        for col in self.sensor_cols:
            if col in df.columns:
                mean_val = df[col].mean()
                std_val = df[col].std()
                cv = (std_val / mean_val * 100) if mean_val != 0 else 0
                
                # 檢查顯著跳變（> 5 個標準差）
                diff = df[col].diff().abs()
                significant_jumps = (diff > std_val * 5).sum() if std_val > 0 else 0
                
                # 評級
                if cv < 1:
                    grade = "⚠️ 極穩定（可能未接觸氣體）"
                elif cv < 5:
                    grade = "✓ 穩定"
                elif cv < 15:
                    grade = "✓ 正常"
                else:
                    grade = "⚠️ 變化較大"
                
                health_report[col] = {
                    'cv_percent': cv,
                    'significant_jumps': significant_jumps,
                    'grade': grade
                }
                
                print(f"      {col}: CV={cv:.2f}%, 顯著跳變={significant_jumps}次, {grade}")
        
        return health_report
    
    def clean_all_data(self, air_data, pineapple_data):
        """批次清洗所有數據"""
        print("="*50)
        print("🧹 開始批次清洗數據")
        print("="*50)
        
        cleaned_air = {}
        cleaned_pineapple = {}
        cleaning_report = {'air': {}, 'pineapple': {}}
        
        # 清洗 air baseline
        for pid, df in air_data.items():
            cleaned_air[pid], cleaning_report['air'][pid] = \
                self.clean_dataset(df, f"Air_{pid}")
        
        # 清洗 pineapple
        for pid, date_dict in pineapple_data.items():
            cleaned_pineapple[pid] = {}
            cleaning_report['pineapple'][pid] = {}
            for date, df in date_dict.items():
                cleaned_pineapple[pid][date], cleaning_report['pineapple'][pid][date] = \
                    self.clean_dataset(df, f"Pineapple_{pid}_{date}")
        
        print("\n✅ 清洗完成！")
        return cleaned_air, cleaned_pineapple, cleaning_report

# 執行清洗
cleaner = DataCleaner()
cleaned_air, cleaned_pineapple, cleaning_report = cleaner.clean_all_data(air_data, pineapple_data)


🧹 開始批次清洗數據

🧹 清洗 Air_01_20250201...
   ⚠️  發現 25 個缺值
   🔬 感測器健康度:
      MQ2_raw: CV=1.64%, 顯著跳變=0次, ✓ 穩定
      MQ3_raw: CV=1.44%, 顯著跳變=0次, ✓ 穩定
      MQ9_raw: CV=1.87%, 顯著跳變=0次, ✓ 穩定
      MQ135_raw: CV=2.02%, 顯著跳變=0次, ✓ 穩定
      TGS2602_raw: CV=0.39%, 顯著跳變=0次, ⚠️ 極穩定（可能未接觸氣體）
   ✓ 原始: 912 rows
   ✓ 移除: 12 rows (1.32%)
   ✓ 保留: 900 rows

🧹 清洗 Air_01_20250202...
   ⚠️  發現 25 個缺值
   🔬 感測器健康度:
      MQ2_raw: CV=3.83%, 顯著跳變=0次, ✓ 穩定
      MQ3_raw: CV=3.56%, 顯著跳變=0次, ✓ 穩定
      MQ9_raw: CV=2.42%, 顯著跳變=0次, ✓ 穩定
      MQ135_raw: CV=2.61%, 顯著跳變=0次, ✓ 穩定
      TGS2602_raw: CV=0.21%, 顯著跳變=0次, ⚠️ 極穩定（可能未接觸氣體）
   ✓ 原始: 912 rows
   ✓ 移除: 12 rows (1.32%)
   ✓ 保留: 900 rows

🧹 清洗 Air_01_20250203...
   ⚠️  發現 25 個缺值
   🔬 感測器健康度:
      MQ2_raw: CV=4.95%, 顯著跳變=0次, ✓ 穩定
      MQ3_raw: CV=2.41%, 顯著跳變=0次, ✓ 穩定
      MQ9_raw: CV=3.92%, 顯著跳變=0次, ✓ 穩定
      MQ135_raw: CV=5.31%, 顯著跳變=0次, ✓ 正常
      TGS2602_raw: CV=0.21%, 顯著跳變=0次, ⚠️ 極穩定（可能未接觸氣體）
   ✓ 原始: 912 rows
   ✓ 移除: 12 rows (1.32%)
   ✓ 保留: 900 rows

🧹 清洗

In [8]:
# ===== Cell 5: Delta（鳳梨 - Air）計算（保留 Arduino 特徵）=====

class DeltaCalculator:
    """計算鳳梨與 air baseline 的差異"""
    
    def __init__(self):
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
        self.env_cols = ['Temp_C', 'Humidity_pct', 'Pressure_hPa']
        
        # 👈 新增：Arduino 已經算好的特徵欄位
        self.arduino_feature_cols = [
            'MQ2_logr', 'MQ3_logr', 'MQ9_logr', 'MQ135_logr', 'TGS2602_logr',
            'MQ2_dlogr', 'MQ3_dlogr', 'MQ9_dlogr', 'MQ135_dlogr', 'TGS2602_dlogr',
            'MQ2_logr_ma5', 'MQ3_logr_ma5', 'MQ9_logr_ma5', 'MQ135_logr_ma5', 'TGS2602_logr_ma5'
        ]
    
    def calculate_delta(self, pineapple_df, air_df):
        """計算單一鳳梨的 delta 值（保留 Arduino 特徵）"""
        # 👈 使用 .copy() 保留所有原始欄位（包含 Arduino 特徵）
        delta_df = pineapple_df.copy()
        
        # 計算 air baseline 的平均值
        air_means = {}
        for col in self.sensor_cols + self.env_cols:
            if col in air_df.columns:
                air_means[col] = air_df[col].mean()
        
        # 計算簡單 delta（鳳梨 - air）用於視覺化比較
        for col in self.sensor_cols:
            if col in air_means:
                delta_df[f'{col}_delta_simple'] = delta_df[col] - air_means[col]
                delta_df[f'{col}_delta_pct'] = (delta_df[col] - air_means[col]) / air_means[col] * 100
        
        # 環境參數也計算 delta（用於監控環境變化）
        for col in self.env_cols:
            if col in air_means:
                delta_df[f'{col}_delta'] = delta_df[col] - air_means[col]
        
        # 👈 新增：標記是否有 Arduino 特徵
        has_arduino_features = all(col in delta_df.columns for col in self.arduino_feature_cols)
        
        # 儲存 baseline 參考值和特徵標記
        delta_df.attrs['air_baseline'] = air_means
        delta_df.attrs['has_arduino_features'] = has_arduino_features
        
        if has_arduino_features:
            print(f"      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）")
        else:
            print(f"      ⚠️  未偵測到完整的 Arduino 特徵")
        
        return delta_df
    
    def calculate_all_deltas(self, cleaned_pineapple, cleaned_air):
        """批次計算所有鳳梨的 delta
        
        cleaned_air 的 key 依命名規則為 "鳳梨編號_日期"
        cleaned_pineapple 為 {pineapple_id: {date: df}}
        """
        print("\n📐 計算 Delta (Pineapple - Air)...")
        
        delta_data = {}
        
        for pid, date_dict in cleaned_pineapple.items():
            delta_data[pid] = {}
            for date, pineapple_df in date_dict.items():
                air_key = f"{pid}_{date}"
                if air_key not in cleaned_air:
                    print(f"   ⚠️  Pineapple_{pid}_{date} 找不到對應的 Air baseline（key={air_key}），跳過")
                    continue
                
                air_df = cleaned_air[air_key]
                print(f"   🔄 Pineapple_{pid}_{date} (使用 Air key={air_key})")
                delta_df = self.calculate_delta(pineapple_df, air_df)
                delta_data[pid][date] = delta_df
        
        print("✅ Delta 計算完成！")
        return delta_data

# 執行 delta 計算
delta_calc = DeltaCalculator()
delta_data = delta_calc.calculate_all_deltas(cleaned_pineapple, cleaned_air)


📐 計算 Delta (Pineapple - Air)...
   🔄 Pineapple_01_20250201 (使用 Air key=01_20250201)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250202 (使用 Air key=01_20250202)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250203 (使用 Air key=01_20250203)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250204 (使用 Air key=01_20250204)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250205 (使用 Air key=01_20250205)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250206 (使用 Air key=01_20250206)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250207 (使用 Air key=01_20250207)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250208 (使用 Air key=01_20250208)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250210 (使用 Air key=01_20250210)
      ✓ 偵測到 Arduino 預處理特徵（log ratio, delta, MA5）
   🔄 Pineapple_01_20250211 (使用 Air key=01_20250211)
      

In [9]:
# ===== Cell 6: 視覺化 - Raw 時序圖 =====

class TimeSeriesVisualizer:
    """時序圖視覺化"""
    
    def __init__(self, output_dir='reports/figures'):
        self.output_dir = output_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
        self.env_cols = ['Temp_C', 'Humidity_pct', 'Pressure_hPa']
        
    def plot_raw_timeseries(self, df, title, save_path=None):
        """繪製 raw 值時序圖（8 subplots）"""
        fig = make_subplots(
            rows=3, cols=3,
            subplot_titles=[
                'MQ2 (Smoke)', 'MQ3 (Alcohol)', 'MQ9 (CO/CH4)',
                'MQ135 (NH3/CO2)', 'TGS2602 (VOC)', 'Temperature',
                'Humidity', 'Pressure', ''
            ],
            vertical_spacing=0.08,
            horizontal_spacing=0.08
        )
        
        time_col = 'timestamp_s' if 'timestamp_s' in df.columns else df.index
        
        # 感測器數據
        sensors = [
            ('MQ2_raw', 1, 1, 'blue'),
            ('MQ3_raw', 1, 2, 'red'),
            ('MQ9_raw', 1, 3, 'green'),
            ('MQ135_raw', 2, 1, 'purple'),
            ('TGS2602_raw', 2, 2, 'orange'),
            ('Temp_C', 2, 3, 'darkred'),
            ('Humidity_pct', 3, 1, 'cyan'),
            ('Pressure_hPa', 3, 2, 'brown')
        ]
        
        for col, row, col_pos, color in sensors:
            if col in df.columns:
                fig.add_trace(
                    go.Scatter(
                        x=df[time_col] if isinstance(time_col, str) else time_col,
                        y=df[col],
                        mode='lines',
                        name=col,
                        line=dict(color=color, width=1.5),
                        showlegend=False
                    ),
                    row=row, col=col_pos
                )
        
        fig.update_xaxes(title_text="Time (s)", row=3)
        fig.update_yaxes(title_text="ADC Value", row=1, col=1)
        fig.update_yaxes(title_text="ADC Value", row=2, col=1)
        fig.update_yaxes(title_text="°C", row=2, col=3)
        fig.update_yaxes(title_text="%", row=3, col=1)
        fig.update_yaxes(title_text="hPa", row=3, col=2)
        
        fig.update_layout(
            title_text=title,
            height=900,
            showlegend=False,
            template='plotly_white'
        )
        
        if save_path:
            fig.write_html(save_path)
            print(f"   💾 已儲存: {save_path}")
        
        return fig
    
    def plot_all_raw_timeseries(self, cleaned_air, cleaned_pineapple):
        """批次繪製所有 raw 時序圖"""
        print("\n📈 繪製 Raw 時序圖...")
        
        # Air baseline
        for pid, df in cleaned_air.items():
            save_path = f"{self.output_dir}/raw_timeseries/air_{pid}_raw.html"
            self.plot_raw_timeseries(df, f"Air Baseline - {pid}", save_path)
        
        # Pineapple
        for pid, date_dict in cleaned_pineapple.items():
            for date, df in date_dict.items():
                save_path = f"{self.output_dir}/raw_timeseries/pineapple_{pid}_{date}_raw.html"
                self.plot_raw_timeseries(df, f"Pineapple {pid} - {date}", save_path)
        
        print("✅ Raw 時序圖繪製完成！")

# 執行視覺化
visualizer = TimeSeriesVisualizer()
visualizer.plot_all_raw_timeseries(cleaned_air, cleaned_pineapple)


📈 繪製 Raw 時序圖...
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250201_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250202_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250203_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250204_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250205_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250206_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250207_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250208_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250210_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20250211_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_01_20260209_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_02_20250202_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_02_20250203_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_02_20250204_raw.html
   💾 已儲存: reports/figures/raw_timeseries/air_02_20250205_ra

In [10]:
# ===== Cell 7: 視覺化 - Delta 時序圖 =====

class DeltaVisualizer:
    """Delta 時序圖視覺化"""
    
    def __init__(self, output_dir='reports/figures'):
        self.output_dir = output_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
    
    def plot_delta_timeseries(self, delta_df, title, save_path=None):
        """繪製 delta 時序圖"""
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=[
                'ΔMQ2', 'ΔMQ3', 'ΔMQ9',
                'ΔMQ135', 'ΔTGS2602', 'Δ% Comparison'
            ],
            specs=[
                [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}],
                [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]
            ],
            vertical_spacing=0.12,
            horizontal_spacing=0.1
        )
        
        time_col = 'timestamp_s' if 'timestamp_s' in delta_df.columns else delta_df.index
        
        colors = ['blue', 'red', 'green', 'purple', 'orange']
        
        # 繪製絕對 delta
        for idx, (col, color) in enumerate(zip(self.sensor_cols, colors)):
            delta_col = f'{col}_delta_simple'
            if delta_col in delta_df.columns:
                row = (idx // 3) + 1
                col_pos = (idx % 3) + 1
                
                fig.add_trace(
                    go.Scatter(
                        x=delta_df[time_col] if isinstance(time_col, str) else time_col,
                        y=delta_df[delta_col],
                        mode='lines',
                        name=col.replace('_raw', ''),
                        line=dict(color=color, width=1.5),
                        showlegend=(idx < 5)
                    ),
                    row=row, col=col_pos
                )
                
                # 加入零線
                fig.add_hline(y=0, line_dash="dash", line_color="gray", 
                             opacity=0.5, row=row, col=col_pos)
        
        # 第 6 個子圖：百分比變化比較
        for col, color in zip(self.sensor_cols, colors):
            pct_col = f'{col}_delta_pct'
            if pct_col in delta_df.columns:
                fig.add_trace(
                    go.Scatter(
                        x=delta_df[time_col] if isinstance(time_col, str) else time_col,
                        y=delta_df[pct_col],
                        mode='lines',
                        name=col.replace('_raw', ''),
                        line=dict(color=color, width=1.5),
                        showlegend=False
                    ),
                    row=2, col=3
                )
        
        fig.add_hline(y=0, line_dash="dash", line_color="gray", 
                     opacity=0.5, row=2, col=3)
        
        # 更新軸標籤
        fig.update_xaxes(title_text="Time (s)", row=2)
        fig.update_yaxes(title_text="Δ ADC", row=1, col=1)
        fig.update_yaxes(title_text="Δ ADC", row=2, col=1)
        fig.update_yaxes(title_text="Δ %", row=2, col=3)
        
        fig.update_layout(
            title_text=title,
            height=700,
            template='plotly_white'
        )
        
        if save_path:
            fig.write_html(save_path)
            print(f"   💾 已儲存: {save_path}")
        
        return fig
    
    def plot_all_delta_timeseries(self, delta_data):
        """批次繪製所有 delta 時序圖"""
        print("\n📉 繪製 Delta 時序圖...")
        
        for pid, date_dict in delta_data.items():
            for date, delta_df in date_dict.items():
                save_path = f"{self.output_dir}/delta_timeseries/pineapple_{pid}_{date}_delta.html"
                self.plot_delta_timeseries(
                    delta_df, 
                    f"Δ (Pineapple - Air) | Pineapple {pid} - {date}", 
                    save_path
                )
        
        print("✅ Delta 時序圖繪製完成！")

# 執行 delta 視覺化
delta_viz = DeltaVisualizer()
delta_viz.plot_all_delta_timeseries(delta_data)


📉 繪製 Delta 時序圖...
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250201_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250202_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250203_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250204_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250205_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250206_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250207_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250208_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250210_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20250211_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_01_20260209_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_02_20250202_delta.html
   💾 已儲存: reports/figures/delta_timeseries/pineapple_02_20250203_delt

In [11]:
# ===== Cell 8: 視覺化 - Arduino 預處理特徵（新增）=====

class ArduinoFeatureVisualizer:
    """Arduino 預處理特徵視覺化"""
    
    def __init__(self, output_dir='reports/figures'):
        self.output_dir = output_dir
        self.sensor_names = ['MQ2', 'MQ3', 'MQ9', 'MQ135', 'TGS2602']
    
    def plot_arduino_features(self, delta_df, title, save_path=None):
        """繪製 Arduino 預處理的特徵（log ratio, delta log ratio, MA5）"""
        
        if not delta_df.attrs.get('has_arduino_features', False):
            print(f"   ⚠️  {title} 沒有 Arduino 特徵，跳過")
            return None
        
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=[
                'Log Ratio (Baseline Normalized)', 'Log Ratio MA5 (Smoothed)',
                'Delta Log Ratio (Rate of Change)', 'Delta Log Ratio Detail',
                'Multi-Sensor Comparison (Log Ratio)', 'Multi-Sensor Comparison (MA5)'
            ],
            vertical_spacing=0.1,
            horizontal_spacing=0.1
        )
        
        time_col = 'timestamp_s' if 'timestamp_s' in delta_df.columns else delta_df.index
        colors = ['blue', 'red', 'green', 'purple', 'orange']
        
        # Row 1: Log Ratio (原始 & MA5)
        for sensor, color in zip(self.sensor_names, colors):
            logr_col = f'{sensor}_logr'
            ma5_col = f'{sensor}_logr_ma5'
            
            if logr_col in delta_df.columns:
                # Log Ratio
                fig.add_trace(
                    go.Scatter(
                        x=delta_df[time_col] if isinstance(time_col, str) else time_col,
                        y=delta_df[logr_col],
                        mode='lines',
                        name=sensor,
                        line=dict(color=color, width=1),
                        legendgroup=sensor,
                        showlegend=True
                    ),
                    row=1, col=1
                )
                
                # MA5
                if ma5_col in delta_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=delta_df[time_col] if isinstance(time_col, str) else time_col,
                            y=delta_df[ma5_col],
                            mode='lines',
                            name=sensor,
                            line=dict(color=color, width=1.5),
                            legendgroup=sensor,
                            showlegend=False
                        ),
                        row=1, col=2
                    )
        
        # Row 2: Delta Log Ratio
        for sensor, color in zip(self.sensor_names, colors):
            dlogr_col = f'{sensor}_dlogr'
            
            if dlogr_col in delta_df.columns:
                # Delta Log Ratio (全圖)
                fig.add_trace(
                    go.Scatter(
                        x=delta_df[time_col] if isinstance(time_col, str) else time_col,
                        y=delta_df[dlogr_col],
                        mode='lines',
                        name=sensor,
                        line=dict(color=color, width=1),
                        legendgroup=sensor,
                        showlegend=False
                    ),
                    row=2, col=1
                )
                
                # Delta Log Ratio Detail (放大看細節)
                fig.add_trace(
                    go.Scatter(
                        x=delta_df[time_col] if isinstance(time_col, str) else time_col,
                        y=delta_df[dlogr_col],
                        mode='markers+lines',
                        name=sensor,
                        line=dict(color=color, width=0.5),
                        marker=dict(size=2),
                        legendgroup=sensor,
                        showlegend=False
                    ),
                    row=2, col=2
                )
        
        # Row 3: Multi-Sensor 綜合比較
        for sensor, color in zip(self.sensor_names, colors):
            logr_col = f'{sensor}_logr'
            ma5_col = f'{sensor}_logr_ma5'
            
            if logr_col in delta_df.columns:
                fig.add_trace(
                    go.Scatter(
                        x=delta_df[time_col] if isinstance(time_col, str) else time_col,
                        y=delta_df[logr_col],
                        mode='lines',
                        name=sensor,
                        line=dict(color=color, width=1, dash='dot'),
                        legendgroup=sensor,
                        showlegend=False
                    ),
                    row=3, col=1
                )
                
                if ma5_col in delta_df.columns:
                    fig.add_trace(
                        go.Scatter(
                            x=delta_df[time_col] if isinstance(time_col, str) else time_col,
                            y=delta_df[ma5_col],
                            mode='lines',
                            name=sensor,
                            line=dict(color=color, width=1.5),
                            legendgroup=sensor,
                            showlegend=False
                        ),
                        row=3, col=2
                    )
        
        # 加入零線
        for row in [1, 2, 3]:
            for col in [1, 2]:
                fig.add_hline(y=0, line_dash="dash", line_color="gray", 
                             opacity=0.3, row=row, col=col)
        
        # 更新軸標籤
        fig.update_xaxes(title_text="Time (s)", row=3)
        fig.update_yaxes(title_text="Log Ratio", row=1, col=1)
        fig.update_yaxes(title_text="Log Ratio (MA5)", row=1, col=2)
        fig.update_yaxes(title_text="Δ Log Ratio", row=2, col=1)
        fig.update_yaxes(title_text="Δ Log Ratio (Detail)", row=2, col=2)
        fig.update_yaxes(title_text="Log Ratio (All)", row=3, col=1)
        fig.update_yaxes(title_text="MA5 (All)", row=3, col=2)
        
        fig.update_layout(
            title_text=f"Arduino Preprocessed Features - {title}",
            height=1000,
            template='plotly_white',
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        if save_path:
            fig.write_html(save_path)
            print(f"   💾 已儲存: {save_path}")
        
        return fig
    
    def plot_all_arduino_features(self, delta_data):
        """批次繪製所有 Arduino 特徵圖"""
        print("\n🎨 繪製 Arduino 特徵圖...")
        
        plotted_count = 0
        for pid, date_dict in delta_data.items():
            for date, delta_df in date_dict.items():
                save_path = f"{self.output_dir}/arduino_features/pineapple_{pid}_{date}_arduino.html"
                result = self.plot_arduino_features(
                    delta_df, 
                    f"Pineapple {pid} - {date}", 
                    save_path
                )
                if result is not None:
                    plotted_count += 1
        
        if plotted_count > 0:
            print(f"✅ Arduino 特徵圖繪製完成！共 {plotted_count} 張")
        else:
            print("⚠️  未找到包含 Arduino 特徵的數據")

# 執行 Arduino 特徵視覺化
arduino_viz = ArduinoFeatureVisualizer()
arduino_viz.plot_all_arduino_features(delta_data)


🎨 繪製 Arduino 特徵圖...
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250201_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250202_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250203_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250204_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250205_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250206_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250207_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250208_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250210_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20250211_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_01_20260209_arduino.html
   💾 已儲存: reports/figures/arduino_features/pineapple_02_20250202_arduino.html
   💾 已儲存: reports/figures/arduino_features/

In [12]:
# ===== Cell 9: 分布與穩定度分析 =====

class DistributionAnalyzer:
    """感測器分布與穩定度分析"""
    
    def __init__(self, output_dir='reports/figures'):
        self.output_dir = output_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
    
    def plot_sensor_distributions(self, data_dict, data_type='pineapple'):
        """繪製感測器數值分布"""
        print(f"\n📊 繪製 {data_type} 分布圖...")
        
        if data_type == 'air':
            # Air: 每個 key 一張圖（key = 鳳梨編號_日期）
            for pid, df in data_dict.items():
                self._plot_single_distribution(df, f"Air_{pid}", 
                    f"{self.output_dir}/distributions/air_{pid}_dist.html")
        else:
            # Pineapple: 每個 ID+date 一張圖
            for pid, date_dict in data_dict.items():
                for date, df in date_dict.items():
                    self._plot_single_distribution(df, f"Pineapple_{pid}_{date}",
                        f"{self.output_dir}/distributions/pineapple_{pid}_{date}_dist.html")
        
        print("✅ 分布圖繪製完成！")
    
    def _plot_single_distribution(self, df, title, save_path):
        """繪製單一數據集的分布"""
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=['MQ2', 'MQ3', 'MQ9', 'MQ135', 'TGS2602', 'Statistics'],
            specs=[[{"type": "box"}, {"type": "box"}, {"type": "box"}],
                   [{"type": "box"}, {"type": "box"}, {"type": "table"}]]
        )
        
        colors = ['blue', 'red', 'green', 'purple', 'orange']
        stats_data = []
        
        for idx, (col, color) in enumerate(zip(self.sensor_cols, colors)):
            if col in df.columns:
                row = (idx // 3) + 1
                col_pos = (idx % 3) + 1
                
                # Box plot
                fig.add_trace(
                    go.Box(
                        y=df[col],
                        name=col.replace('_raw', ''),
                        marker_color=color,
                        boxmean='sd',
                        showlegend=False
                    ),
                    row=row, col=col_pos
                )
                
                # 計算統計量
                stats_data.append([
                    col.replace('_raw', ''),
                    f"{df[col].mean():.2f}",
                    f"{df[col].std():.2f}",
                    f"{df[col].min():.0f}",
                    f"{df[col].max():.0f}",
                    f"{df[col].std()/df[col].mean()*100:.2f}%"
                ])
        
        # 統計表格
        fig.add_trace(
            go.Table(
                header=dict(
                    values=['Sensor', 'Mean', 'Std', 'Min', 'Max', 'CV%'],
                    fill_color='lightgray',
                    align='left'
                ),
                cells=dict(
                    values=list(zip(*stats_data)),
                    fill_color='white',
                    align='left'
                )
            ),
            row=2, col=3
        )
        
        fig.update_layout(
            title_text=f"Sensor Distributions - {title}",
            height=600,
            showlegend=False,
            template='plotly_white'
        )
        
        fig.write_html(save_path)
    
    def calculate_stability_scores(self, data_dict, data_type='pineapple'):
        """計算穩定度分數（CV: Coefficient of Variation）"""
        print(f"\n🎯 計算 {data_type} 穩定度分數...")
        
        stability_report = {}
        
        if data_type == 'air':
            for pid, df in data_dict.items():
                stability_report[pid] = self._calc_single_stability(df)
        else:
            for pid, date_dict in data_dict.items():
                stability_report[pid] = {}
                for date, df in date_dict.items():
                    stability_report[pid][date] = self._calc_single_stability(df)
        
        return stability_report
    
    def _calc_single_stability(self, df):
        """計算單一數據集的穩定度"""
        stability = {}
        
        for col in self.sensor_cols:
            if col in df.columns:
                mean_val = df[col].mean()
                std_val = df[col].std()
                cv = (std_val / mean_val * 100) if mean_val != 0 else np.inf
                
                stability[col] = {
                    'mean': mean_val,
                    'std': std_val,
                    'cv_percent': cv,
                    'stability_grade': self._grade_stability(cv)
                }
        
        return stability
    
    def _grade_stability(self, cv):
        """根據 CV 評分穩定度"""
        if cv < 5:
            return 'A (Excellent)'
        elif cv < 10:
            return 'B (Good)'
        elif cv < 20:
            return 'C (Fair)'
        else:
            return 'D (Poor)'

# 執行分布分析
dist_analyzer = DistributionAnalyzer()
dist_analyzer.plot_sensor_distributions(cleaned_air, 'air')
dist_analyzer.plot_sensor_distributions(cleaned_pineapple, 'pineapple')

# 計算穩定度
air_stability = dist_analyzer.calculate_stability_scores(cleaned_air, 'air')
pineapple_stability = dist_analyzer.calculate_stability_scores(cleaned_pineapple, 'pineapple')


📊 繪製 air 分布圖...
✅ 分布圖繪製完成！

📊 繪製 pineapple 分布圖...
✅ 分布圖繪製完成！

🎯 計算 air 穩定度分數...

🎯 計算 pineapple 穩定度分數...


In [13]:
# ===== Cell 10: 相關性分析 =====

class CorrelationAnalyzer:
    """感測器間相關性分析"""
    
    def __init__(self, output_dir='reports/figures'):
        self.output_dir = output_dir
        self.sensor_cols = ['MQ2_raw', 'MQ3_raw', 'MQ9_raw', 'MQ135_raw', 'TGS2602_raw']
    
    def plot_correlation_matrix(self, df, title, save_path):
        """繪製相關性矩陣熱力圖"""
        # 只選擇感測器欄位
        sensor_data = df[self.sensor_cols]
        
        # 計算相關係數
        corr_matrix = sensor_data.corr()
        
        # 繪製熱力圖
        fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=[col.replace('_raw', '') for col in corr_matrix.columns],
            y=[col.replace('_raw', '') for col in corr_matrix.index],
            colorscale='RdBu',
            zmid=0,
            text=corr_matrix.values,
            texttemplate='%{text:.2f}',
            textfont={"size": 12},
            colorbar=dict(title="Correlation")
        ))
        
        fig.update_layout(
            title=f"Sensor Correlation Matrix - {title}",
            xaxis_title="Sensors",
            yaxis_title="Sensors",
            height=500,
            width=600,
            template='plotly_white'
        )
        
        fig.write_html(save_path)
        print(f"   💾 已儲存: {save_path}")
    
    def analyze_all_correlations(self, cleaned_air, cleaned_pineapple):
        """批次分析所有相關性"""
        print("\n🔗 繪製相關性矩陣...")
        
        # Air
        for pid, df in cleaned_air.items():
            save_path = f"{self.output_dir}/correlation/air_{pid}_corr.html"
            self.plot_correlation_matrix(df, f"Air_{pid}", save_path)
        
        # Pineapple
        for pid, date_dict in cleaned_pineapple.items():
            for date, df in date_dict.items():
                save_path = f"{self.output_dir}/correlation/pineapple_{pid}_{date}_corr.html"
                self.plot_correlation_matrix(df, f"Pineapple_{pid}_{date}", save_path)
        
        print("✅ 相關性分析完成！")

# 執行相關性分析
corr_analyzer = CorrelationAnalyzer()
corr_analyzer.analyze_all_correlations(cleaned_air, cleaned_pineapple)


🔗 繪製相關性矩陣...
   💾 已儲存: reports/figures/correlation/air_01_20250201_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250202_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250203_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250204_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250205_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250206_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250207_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250208_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250210_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20250211_corr.html
   💾 已儲存: reports/figures/correlation/air_01_20260209_corr.html
   💾 已儲存: reports/figures/correlation/air_02_20250202_corr.html
   💾 已儲存: reports/figures/correlation/air_02_20250203_corr.html
   💾 已儲存: reports/figures/correlation/air_02_20250204_corr.html
   💾 已儲存: reports/figures/correlation/air_02_20250205_corr.html
   💾 已儲存: reports/figures/

In [14]:
# ===== Cell 11: 綜合摘要報告 =====

class SummaryReportGenerator:
    """生成每顆鳳梨的摘要報告"""
    
    def __init__(self, output_dir='reports/summary'):
        self.output_dir = output_dir
    
    def generate_full_report(self, cleaned_air, cleaned_pineapple, delta_data, 
                            cleaning_report, air_stability, pineapple_stability):
        """生成完整的摘要報告"""
        print("\n📝 生成摘要報告...")
        
        report = {
            'generation_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'overview': self._generate_overview(cleaned_air, cleaned_pineapple),
            'data_quality': self._generate_quality_report(cleaning_report),
            'stability_analysis': self._generate_stability_report(air_stability, pineapple_stability),
            'sensor_health': self._generate_sensor_health_summary(cleaning_report),  # 👈 新增
            'arduino_features': self._check_arduino_features(delta_data),  # 👈 新增
            'recommendations': self._generate_recommendations(cleaned_air, cleaned_pineapple, 
                                                             pineapple_stability, cleaning_report)
        }
        
        # 儲存 JSON
        json_path = f"{self.output_dir}/full_report.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(report, f, ensure_ascii=False, indent=2, default=str)
        
        # 生成人類可讀的報告
        self._generate_readable_report(report)
        
        print(f"✅ 報告已生成: {json_path}")
        return report
    
    def _generate_overview(self, cleaned_air, cleaned_pineapple):
        """生成總覽"""
        total_pineapples = len(cleaned_pineapple)
        total_measurements = sum(len(date_dict) for date_dict in cleaned_pineapple.values())
        
        return {
            'total_pineapples': total_pineapples,
            'total_measurements': total_measurements,
            'pineapple_ids': list(cleaned_pineapple.keys())
        }
    
    def _generate_quality_report(self, cleaning_report):
        """生成數據品質報告"""
        quality = {'air': {}, 'pineapple': {}}
        
        for data_type in ['air', 'pineapple']:
            for pid, info in cleaning_report[data_type].items():
                if data_type == 'air':
                    quality['air'][pid] = {
                        'data_retention_rate': f"{(1 - info['removed_rows']/info['original_rows'])*100:.2f}%",
                        'total_outliers': sum(info['outlier_counts'].values()) if info['outlier_counts'] else 0
                    }
                else:
                    quality['pineapple'][pid] = {}
                    for date, date_info in info.items():
                        quality['pineapple'][pid][date] = {
                            'data_retention_rate': f"{(1 - date_info['removed_rows']/date_info['original_rows'])*100:.2f}%",
                            'total_outliers': sum(date_info['outlier_counts'].values()) if date_info['outlier_counts'] else 0
                        }
        
        return quality
    
    def _generate_stability_report(self, air_stability, pineapple_stability):
        """生成穩定度報告"""
        return {
            'air': air_stability,
            'pineapple': pineapple_stability
        }
    
    def _generate_sensor_health_summary(self, cleaning_report):
        """👈 新增：生成感測器健康度摘要"""
        health_summary = {'air': {}, 'pineapple': {}}
        
        for data_type in ['air', 'pineapple']:
            for pid, info in cleaning_report[data_type].items():
                if data_type == 'air':
                    health_summary['air'][pid] = info.get('sensor_health', {})
                else:
                    health_summary['pineapple'][pid] = {}
                    for date, date_info in info.items():
                        health_summary['pineapple'][pid][date] = date_info.get('sensor_health', {})
        
        return health_summary
    
    def _check_arduino_features(self, delta_data):
        """👈 新增：檢查 Arduino 特徵可用性"""
        feature_status = {}
        
        for pid, date_dict in delta_data.items():
            feature_status[pid] = {}
            for date, delta_df in date_dict.items():
                has_features = delta_df.attrs.get('has_arduino_features', False)
                feature_status[pid][date] = has_features
        
        total = sum(len(dates) for dates in feature_status.values())
        available = sum(1 for dates in feature_status.values() for has in dates.values() if has)
        
        return {
            'availability': f"{available}/{total}",
            'percentage': f"{available/total*100:.1f}%" if total > 0 else "0%",
            'details': feature_status
        }
    
    def _generate_recommendations(self, cleaned_air, cleaned_pineapple, 
                                 pineapple_stability, cleaning_report):
        """生成建議（含感測器健康度）"""
        recommendations = []
        
        # 檢查數據量
        total_pineapples = len(cleaned_pineapple)
        if total_pineapples < 10:
            recommendations.append({
                'type': 'data_sufficiency',
                'priority': 'high',
                'message': f"目前只有 {total_pineapples} 顆鳳梨，建議追加至少 10 顆以上以提高模型可靠性"
            })
        
        # 檢查穩定度
        for pid, date_dict in pineapple_stability.items():
            for date, stability in date_dict.items():
                poor_sensors = [sensor for sensor, info in stability.items() 
                               if 'Poor' in info['stability_grade']]
                if poor_sensors:
                    recommendations.append({
                        'type': 'sensor_stability',
                        'priority': 'medium',
                        'pineapple': f"{pid}_{date}",
                        'message': f"感測器 {poor_sensors} 穩定度較差（CV > 20%）"
                    })
        
        # 👈 新增：檢查感測器健康度
        for pid, info in cleaning_report['pineapple'].items():
            for date, date_info in info.items():
                health = date_info.get('sensor_health', {})
                for sensor, sensor_health in health.items():
                    if sensor_health['cv_percent'] < 1:
                        recommendations.append({
                            'type': 'sensor_health',
                            'priority': 'low',
                            'pineapple': f"{pid}_{date}",
                            'message': f"{sensor} 變異極小（CV < 1%），可能未接觸氣體或感測器故障"
                        })
                    
                    if sensor_health['significant_jumps'] > 10:
                        recommendations.append({
                            'type': 'sensor_noise',
                            'priority': 'medium',
                            'pineapple': f"{pid}_{date}",
                            'message': f"{sensor} 有 {sensor_health['significant_jumps']} 次顯著跳變，檢查訊號品質"
                        })
        
        return recommendations
    
    def _generate_readable_report(self, report):
        """生成人類可讀的 Markdown 報告"""
        md_path = f"{self.output_dir}/summary_report.md"
        
        with open(md_path, 'w', encoding='utf-8') as f:
            f.write("# 鳳梨氣味感測器數據分析報告\n\n")
            f.write(f"**生成時間**: {report['generation_time']}\n\n")
            
            f.write("## 📊 數據總覽\n\n")
            f.write(f"- 鳳梨總數: {report['overview']['total_pineapples']} 顆\n")
            f.write(f"- 測量次數: {report['overview']['total_measurements']} 次\n")
            f.write(f"- 鳳梨編號: {', '.join(report['overview']['pineapple_ids'])}\n\n")
            
            # 👈 新增：Arduino 特徵可用性
            f.write("## 🎨 Arduino 預處理特徵\n\n")
            f.write(f"- 可用性: {report['arduino_features']['availability']} ({report['arduino_features']['percentage']})\n")
            if report['arduino_features']['percentage'] == '100.0%':
                f.write("- ✅ 所有數據集都包含完整的 Arduino 特徵（log ratio, delta, MA5）\n\n")
            else:
                f.write("- ⚠️  部分數據集缺少 Arduino 特徵\n\n")
            
            f.write("## ✅ 數據品質評估\n\n")
            f.write("### Air Baseline\n\n")
            for pid, quality in report['data_quality']['air'].items():
                f.write(f"- **Air_{pid}**: 保留率 {quality['data_retention_rate']}, "
                       f"異常值 {quality['total_outliers']} 個\n")
            
            f.write("\n### Pineapple 測量\n\n")
            for pid, dates in report['data_quality']['pineapple'].items():
                for date, quality in dates.items():
                    f.write(f"- **Pineapple_{pid}_{date}**: 保留率 {quality['data_retention_rate']}, "
                           f"異常值 {quality['total_outliers']} 個\n")
            
            f.write("\n## 💡 建議事項\n\n")
            if report['recommendations']:
                # 按優先級分組
                high_priority = [r for r in report['recommendations'] if r['priority'] == 'high']
                medium_priority = [r for r in report['recommendations'] if r['priority'] == 'medium']
                low_priority = [r for r in report['recommendations'] if r['priority'] == 'low']
                
                if high_priority:
                    f.write("### 🔴 高優先級\n\n")
                    for rec in high_priority:
                        f.write(f"- **{rec['type']}**: {rec['message']}\n")
                    f.write("\n")
                
                if medium_priority:
                    f.write("### 🟡 中優先級\n\n")
                    for rec in medium_priority:
                        pineapple = rec.get('pineapple', '')
                        f.write(f"- **{rec['type']}** ({pineapple}): {rec['message']}\n")
                    f.write("\n")
                
                if low_priority:
                    f.write("### 🟢 低優先級\n\n")
                    for rec in low_priority:
                        pineapple = rec.get('pineapple', '')
                        f.write(f"- **{rec['type']}** ({pineapple}): {rec['message']}\n")
                    f.write("\n")
            else:
                f.write("✅ 數據品質良好，無需額外處理\n\n")
        
        print(f"   💾 已生成 Markdown 報告: {md_path}")

# 生成報告
report_gen = SummaryReportGenerator()
final_report = report_gen.generate_full_report(
    cleaned_air, cleaned_pineapple, delta_data,
    cleaning_report, air_stability, pineapple_stability
)


📝 生成摘要報告...
   💾 已生成 Markdown 報告: reports/summary/summary_report.md
✅ 報告已生成: reports/summary/full_report.json


In [15]:
# ===== Cell 12: 最終總結 =====

print("\n" + "="*60)
print("🎉 Step 1: 自動化資料清洗與初步分析 (EDA) 完成！")
print("="*60)

print("\n📁 已生成的檔案：")
print("   📊 Raw 時序圖: reports/figures/raw_timeseries/")
print("   📉 Delta 時序圖: reports/figures/delta_timeseries/")
print("   🎨 Arduino 特徵圖: reports/figures/arduino_features/")
print("   📈 分布圖: reports/figures/distributions/")
print("   🔗 相關性矩陣: reports/figures/correlation/")
print("   📝 摘要報告: reports/summary/")

print("\n💡 關鍵發現：")
print(f"   • 鳳梨總數: {final_report['overview']['total_pineapples']} 顆")
print(f"   • 測量次數: {final_report['overview']['total_measurements']} 次")
print(f"   • Arduino 特徵可用性: {final_report['arduino_features']['percentage']}")

if final_report['recommendations']:
    print("\n⚠️  建議事項：")
    high_priority = [r for r in final_report['recommendations'] if r['priority'] == 'high']
    medium_priority = [r for r in final_report['recommendations'] if r['priority'] == 'medium']
    
    for rec in high_priority[:3]:  # 只顯示前3個高優先級
        print(f"   [高] {rec['message']}")
    
    for rec in medium_priority[:3]:  # 只顯示前3個中優先級
        print(f"   [中] {rec['message']}")
    
    if len(final_report['recommendations']) > 6:
        print(f"   ... 等共 {len(final_report['recommendations'])} 項建議（詳見報告）")

print("\n✅ 數據可用性評估：")
if final_report['overview']['total_pineapples'] >= 10:
    print("   ✓ 數據量充足，可進入 Step 3（成熟階段定義）")
else:
    print("   ⚠️  建議追加測試（Step 2），目標至少 10 顆鳳梨")

print("\n🚀 下一步 (Step 2)：")
print("   1. 檢視 reports/ 資料夾中的所有圖表")
print("   2. 特別注意 Arduino 特徵圖（arduino_features/）")
print("   3. 與小組討論數據品質與樣本數量")
print("   4. 決定是否需要追加測試")
print("   5. 確認後進入 Step 3: Change-point detection & Clustering")

print("\n📌 提醒：")
print("   • Arduino 已預處理的特徵（log ratio, delta, MA5）已完整保留")
print("   • 感測器健康度報告已整合到摘要中")
print("   • 所有視覺化圖表都是互動式 HTML，可用瀏覽器開啟")

print("\n" + "="*60)


🎉 Step 1: 自動化資料清洗與初步分析 (EDA) 完成！

📁 已生成的檔案：
   📊 Raw 時序圖: reports/figures/raw_timeseries/
   📉 Delta 時序圖: reports/figures/delta_timeseries/
   🎨 Arduino 特徵圖: reports/figures/arduino_features/
   📈 分布圖: reports/figures/distributions/
   🔗 相關性矩陣: reports/figures/correlation/
   📝 摘要報告: reports/summary/

💡 關鍵發現：
   • 鳳梨總數: 11 顆
   • 測量次數: 76 次
   • Arduino 特徵可用性: 98.7%

⚠️  建議事項：
   [中] MQ2_raw 有 48 次顯著跳變，檢查訊號品質
   [中] MQ9_raw 有 52 次顯著跳變，檢查訊號品質
   [中] MQ9_raw 有 14 次顯著跳變，檢查訊號品質
   ... 等共 255 項建議（詳見報告）

✅ 數據可用性評估：
   ✓ 數據量充足，可進入 Step 3（成熟階段定義）

🚀 下一步 (Step 2)：
   1. 檢視 reports/ 資料夾中的所有圖表
   2. 特別注意 Arduino 特徵圖（arduino_features/）
   3. 與小組討論數據品質與樣本數量
   4. 決定是否需要追加測試
   5. 確認後進入 Step 3: Change-point detection & Clustering

📌 提醒：
   • Arduino 已預處理的特徵（log ratio, delta, MA5）已完整保留
   • 感測器健康度報告已整合到摘要中
   • 所有視覺化圖表都是互動式 HTML，可用瀏覽器開啟

